# Get hydrology results from Stormwater Heatmap Modeling 

This notebook demonstrates how to download HSPF modeling results from BigQuery

We will use the pandas_gbq package to interact with BigQuery. See https://googleapis.dev/python/pandas-gbq/latest/ for more info. 

In [14]:
import pandas as pd
import pandas_gbq

## Import Libraries

In [15]:
# Set your default project here
pandas_gbq.context.project = 'tnc-data-v1'
pandas_gbq.context.dialect = 'standard'

## Build Query  

### Input Query Parameters

In [29]:
table = "`tnc-data-v1.hydrology.gfdl`" #table where results are 
hru = "hru000" #hru code 
year_0 = 1970 #beginning year 
year_n = 1999 #ending year 
comps = '("suro", "ifwo")' #components 
    #- available options are:  
        #suro (surface runoff), 
        #ifwo (interflow), and 
        #agwo (groundwater outflow) - generally not used for stormwater results
grid = "'ID15_V5'" #WRF grid id 

**Bonus:** How to find the grid id for a lat, lon point

In [16]:
#example sql statment
sql = """select id  
from 
geometry.cigGeo
where 
ST_DWithin(
  st_geogpoint(-122.4443, 47.2529)
  , polygon, 0) 
  """

#--not run (uncomment to run)
#send statement 
# pandas_gbq.read_gbq(sql,project_id='tnc-data-v1')
# returns: 
# > 15_V5

## Query Statement

In [31]:
sql = """
SELECT
  Datetime,
  SUM({}) AS mm_per_hour
FROM
  {}
WHERE
  year BETWEEN {}
  AND {}
  AND comp in {}
  AND grid = {}
GROUP BY
  Datetime
ORDER BY
  Datetime
""".format(hru,table,year_0,year_n,comps,grid)


# Get results

In [32]:
pandas_gbq.read_gbq(sql,project_id='tnc-data-v1')

/opt/miniconda3/envs/stormpiper/lib/python3.8/site-packages/google/cloud/bigquery/table.py:1939: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


,Datetime,mm_per_hour
0,1970-01-02 00:00:00+00:00,0.0
1,1970-01-02 01:00:00+00:00,0.0
2,1970-01-02 02:00:00+00:00,0.0
3,1970-01-02 03:00:00+00:00,0.0
4,1970-01-02 04:00:00+00:00,0.0
...,...,...
262939,1999-12-31 19:00:00+00:00,0.0
262940,1999-12-31 20:00:00+00:00,0.0
262941,1999-12-31 21:00:00+00:00,0.0
262942,1999-12-31 22:00:00+00:00,0.0
